In [1]:
import json
import re
import tempfile
from collections import defaultdict
from dataclasses import dataclass, field
from io import BytesIO
from pathlib import Path
from typing import Literal
from urllib.request import urlopen

from graphviz import Digraph
from PIL import Image, ImageDraw, ImageEnhance, ImageFont, ImageFilter
from slpp import slpp as lua

In [2]:
OUTPUT_DIR_PATH = Path("output")
FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"

In [3]:
# Configuration for the Factorio game
DIFFICULTY = "normal"

# Define the machines to use
CRAFTING_MACHINE = "assembling-machine-3"
SMELTING_MACHINE = "electric-furnace"

SMELTING_SPEED = {
    "stone-furnace": 1.0,
    "steel-furnace": 2.0,
    "electric-furnace": 2.0,
}.get(SMELTING_MACHINE, 2.0)

CRAFTING_SPEED = {
    "assembling-machine-1": 0.5,
    "assembling-machine-2": 0.75,
    "assembling-machine-3": 1.25,
}.get(CRAFTING_MACHINE, 1.25)

RECIPE_SPEEDS = {
    "advanced-crafting": CRAFTING_SPEED,
    "crafting": CRAFTING_SPEED,
    "crafting-with-fluid": CRAFTING_SPEED,
    "smelting": SMELTING_SPEED,
}

In [4]:
# FATORIO DATA REPOSITORY
FACTORIO_VERSION = "2.0.71"

FACTORIO_DATA_PATH = Path("/mnt/c/Program Files (x86)/Steam/steamapps/common/Factorio/data")
FACTORIO_GITHUB_URL = f"https://raw.githubusercontent.com/wube/factorio-data/refs/tags/{FACTORIO_VERSION}"

RECIPE_FILE_PATH = "base/prototypes/recipe.lua"
ITEM_FILE_PATH = "base/prototypes/item.lua"
FLUID_FILE_PATH = "base/prototypes/fluid.lua"
EQUIPMENT_FILE_PATH = "base/prototypes/equipment.lua"

def read_factorio_file(file_path: str, binary: bool = False) -> str | bytes:
    """
    Returns the contents of a Factorio data file from local installation or GitHub.

    Args:
        file_path: relative path inside Factorio folder
        binary: if True, returns bytes instead of a string

    Returns:
        str (text) or bytes (binary)
    """
    file_path = file_path.replace("__base__", "base")
    # Try local installation first
    local_path = FACTORIO_DATA_PATH / file_path
    if local_path.exists():
        if binary:
            return local_path.read_bytes()
        else:
            return local_path.read_text(encoding="utf-8")
    
    # Fallback to GitHub (only has text files)
    url = f"{FACTORIO_GITHUB_URL}/{file_path}"
    with urlopen(url) as response:
        data = response.read()
        if binary:
            return data
        return data.decode("utf-8")

def clean_factorio_lua(text: str) -> str:
    """Clean Factorio Lua file by removing comments and unnecessary parts."""
    # Remove multi-line comments
    text = re.sub(r"--\[\[.*?\]\]", "", text, flags=re.DOTALL)
    # Remove single-line comments
    text = re.sub(r"--.*", "", text)
    return text

# Clean recipes

In [5]:
recipe_file_text = read_factorio_file(RECIPE_FILE_PATH)

# Remove wrapper
text = recipe_file_text.split("data:extend\n(",1)[1].removesuffix(")")
text = clean_factorio_lua(text)
raw_recipes = lua.decode(text)

(OUTPUT_DIR_PATH / "recipe.json").write_text(json.dumps(raw_recipes, indent=2))

print(len(raw_recipes))


192


In [6]:
@dataclass
class Recipe:
    name: str
    category: str = "crafting"
    energy_required: float = 0.5
    ingredients: dict[str, int] = field(default_factory=dict)
    result_amount: float = 1

    @property
    def time_per_cycle(self) -> float:
        """Time per crafting cycle adjusted by machine speed."""
        return self.energy_required / RECIPE_SPEEDS.get(self.category, CRAFTING_SPEED)
    
    @property
    def machine(self) -> str:
        if "crafting" in self.category:
            return CRAFTING_MACHINE
        if self.category == "smelting":
            return SMELTING_MACHINE
        if self.category == "oil-processing":
            return "oil-refinery"
        if self.category == "chemistry":
            return "chemical-plant"
        if self.category == "rocket-building":
            return "rocket-silo"
        if self.category == "centrifuging":
            return "centrifuge"
        print(f"Couldn't determine name for recipe {self.name} with category {self.category}")

recipes: dict[str, Recipe] = {}
for raw_recipe in raw_recipes:
    name = raw_recipe["name"]
    amount_dict = raw_recipe.get(DIFFICULTY, raw_recipe)

    # Parse out the ingredients
    ingredients: dict[str, int] = {}
    raw_ingredients = amount_dict.get("ingredients")
    ingredients = {raw_ingredient["name"]: raw_ingredient["amount"] for raw_ingredient in raw_ingredients}

    # Parse out the results
    raw_results = amount_dict.get("results")
    if len(raw_results) != 1:
        # Skip oil and uranium recipes for now
        print(f"Skipping {name} with multiple results")
        continue

    raw_result_dict = next(iter(raw_results))
    result_name = raw_result_dict["name"]
    result_amount = raw_result_dict["amount"]
    if result_name != name:
        # Skip oil and uranium recipes for now
        print(f"Skipping {name} with different result name {result_name}")
        continue

    recipes[name] = Recipe(
        name=name,
        category=raw_recipe.get("category", "crafting"),
        ingredients=ingredients,
        energy_required=amount_dict.get("energy_required", 0.5),
        result_amount=result_amount,
    )


Skipping basic-oil-processing with different result name petroleum-gas
Skipping advanced-oil-processing with multiple results
Skipping coal-liquefaction with multiple results
Skipping heavy-oil-cracking with different result name light-oil
Skipping light-oil-cracking with different result name petroleum-gas
Skipping solid-fuel-from-light-oil with different result name solid-fuel
Skipping solid-fuel-from-petroleum-gas with different result name solid-fuel
Skipping solid-fuel-from-heavy-oil with different result name solid-fuel
Skipping uranium-processing with multiple results
Skipping kovarex-enrichment-process with multiple results
Skipping nuclear-fuel-reprocessing with different result name uranium-238


# Clean item, fluid, and equipment icon paths

In [7]:
recipe_file_text = read_factorio_file(ITEM_FILE_PATH)

# Remove wrapper
text = recipe_file_text.split("data:extend\n(",1)[1].removesuffix(")\n")
text = clean_factorio_lua(text)
raw_items = lua.decode(text)

item_icon_paths: dict[str, str] = {
    raw_item.get("name"): raw_item.get("icon")
    for raw_item in raw_items
}
len(item_icon_paths)

233

In [8]:
fluid_file_text = read_factorio_file(FLUID_FILE_PATH)

# Remove wrapper
text = fluid_file_text.split("data:extend(",2)[2].removesuffix(")\n")
text = clean_factorio_lua(text)
raw_fluids = lua.decode(text)

fluid_icon_paths: dict[str, str] = {
    raw_fluid.get("name"): raw_fluid.get("icon")
    for raw_fluid in raw_fluids
}
len(fluid_icon_paths)

8

In [ ]:
equipment_file_text = read_factorio_file(EQUIPMENT_FILE_PATH)

# Remove wrapper
text = equipment_file_text.split("data:extend(",1)[1].removesuffix(")\n")
text = clean_factorio_lua(text)
raw_equipments = lua.decode(text)

equipment_icon_paths: dict[str, str] = {
    raw_equipment.get("name"): raw_equipment.get("icon")
    for raw_equipment in raw_equipments
}
len(equipment_icon_paths)

14

In [94]:
RESOLUTIONS = Literal[64, 32, 16, 8]

def get_icon_file_path(product: str) -> str | None:
    """Get icon file path from local installation or GitHub."""
    return item_icon_paths.get(product) or fluid_icon_paths.get(product) or equipment_icon_paths.get(product)

def read_icon_file(product: str, resolution: RESOLUTIONS=64) -> Image.Image | None:
    """Read icon file from local installation or GitHub."""
    icon_path = get_icon_file_path(product)
    if not icon_path:
        print(f"No icon path found for product: {product}")
        return None
    icon_bytes = read_factorio_file(icon_path, binary=True)
    img = Image.open(BytesIO(icon_bytes))
    if img.size != (120, 64):
        raise ValueError(f"Unexpected icon size for {product}: {img.size}")
    if resolution == 64:
        return img.crop((0, 0, 64, 64)).convert("RGBA")
    elif resolution == 32:
        return img.crop((64, 0, 64+32, 32)).convert("RGBA")
    elif resolution == 16:
        return img.crop((64+32, 0, 64+32+16, 16))
    elif resolution == 8:
        return img.crop((64+32+16, 0, 64+32+16+8, 8))
    else:
        raise ValueError(f"Unsupported resolution: {resolution}")

def create_product_image(
        product: str,
        machine: str | None,
        production_rate: float,
        machine_count: float,
        save_dir: Path=OUTPUT_DIR_PATH,
) -> str | None:
    """Create an image showing a product with its machine and production rate."""
    # Load images
    machine_img = read_icon_file(machine, resolution=64) if machine else None
    product_img = read_icon_file(product, resolution=64)

    if product_img is None:
        return None

    if machine_count and machine_img:
        # Upscale
        image = machine_img.resize((128, 128), resample=Image.BICUBIC)
        
        # Create shadow
        alpha = product_img.split()[3]
        shadow = Image.new("RGBA", product_img.size, (0, 0, 0, 255))
        shadow.putalpha(alpha)

        padding = 10
        w, h = shadow.size
        shadow_padded = Image.new("RGBA", (w + padding * 2, h + padding * 2), (0, 0, 0, 0))
        shadow_padded.paste(shadow, (0, 0), shadow)
        shadow_padded = shadow_padded.filter(ImageFilter.GaussianBlur(radius=3))

        # Combine images
        image.paste(shadow_padded, (0,0), shadow_padded)
        image.paste(product_img, (0, 0), product_img)
        
        quantity_text = f"{machine_count:.3g}"

    else:
        image = product_img.resize((128, 128), resample=Image.BICUBIC)
        quantity_text = ""

    # Draw text
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype(FONT_PATH, size=18)
    draw = ImageDraw.Draw(image)
    bbox = draw.textbbox((0, 0), quantity_text, font=font)
    text_w_machine = bbox[2] - bbox[0]
    text_h_machine = bbox[3] - bbox[1]
    draw.text((125 - text_w_machine, 120 - text_h_machine), quantity_text, fill="white", font=font, stroke_width=3, stroke_fill="black")

    production_text = f"{production_rate:.3g}/s"
    bbox_prod = draw.textbbox((0, 0), production_text, font=font)
    text_w_prod = bbox_prod[2] - bbox_prod[0]
    draw.text((125-text_w_prod, 0), production_text, fill=(200, 200, 200, 255), font=font, stroke_width=3, stroke_fill="black")

    # Save
    save_file = save_dir / f"{product}_with_{machine}_{production_rate:.3g}_{quantity_text}.png"  
    image.save(save_file)
    return str(save_file)

create_product_image(
    product="automation-science-pack",
    machine = "assembling-machine-3",
    machine_count=10,
    production_rate=12.5
)

'output/automation-science-pack_with_assembling-machine-3_12.5_10.png'

# Create graph

In [95]:
# Get topological order of products so we can calculate demands correctly

def get_topological_order(recipes: dict[str, Recipe], target: str, raw_materials=set()) -> list[str]:
    """Get a topological ordering of products via DFS."""
    visited = set()
    topo_order = []

    def dfs(product: str):
        if product in visited:
            return
        visited.add(product)
        recipe = recipes.get(product)
        is_raw_material = product in raw_materials or recipe is None
        if not is_raw_material:
            for ing in recipe.ingredients:
                dfs(ing)
        topo_order.append(product)

    dfs(target)
    return topo_order

In [97]:
def build_blueprint_graph(recipes: dict[str, Recipe], target: str, target_demand: float=2.5, raw_materials=set()):
    """Build a graph where edges show how many machines and items are required."""
    # Get topological ordering of nodes via DFS
    topo_order = get_topological_order(recipes, target, raw_materials)

    # compute item demand and machine counts
    target_recipe = recipes[target]
    item_demands = defaultdict(float, {target: target_demand})
    machine_counts = defaultdict(float, {target: target_demand / target_recipe.result_amount * target_recipe.time_per_cycle})
    edges: list[tuple[str, str, float, float]] = []
    for product in reversed(topo_order):
        recipe = recipes.get(product)
        if product in raw_materials or recipe is None:
            continue  # raw material, no inputs
        for ing in recipe.ingredients:
            item_demand_edge = machine_counts[product] * recipe.ingredients[ing] / recipe.time_per_cycle
            item_demands[ing] += item_demand_edge
            machine_count_edge = 0.0
            ing_recipe = recipes.get(ing)
            if ing_recipe:
                machine_count_edge = item_demand_edge / ing_recipe.result_amount * ing_recipe.time_per_cycle
            machine_counts[ing] += machine_count_edge
            edges.append((ing, product, machine_count_edge, item_demand_edge))

    # Build graphviz graph
    dot = Digraph()

    with tempfile.TemporaryDirectory() as tmpdir:
        # Nodes
        for product in topo_order:
            recipe = recipes.get(product)
            icon_path = create_product_image(
                product=product,
                machine=recipe.machine if recipe else None,
                production_rate=item_demands[product],
                machine_count=machine_counts.get(product, 0.0),
                save_dir=Path(tmpdir),
            )
            
            if icon_path is not None and Path(icon_path).exists():
                dot.node(
                    product,
                    label="",
                    image=str(icon_path),
                    shape="none",
                    imagescale="true"
                )
            else:
                dot.node(product, label=f"{product} - {item_demands[product]:.3g}/s - {machine_counts.get(product, 0.0):.3g} machines")

        # Edges
        for ing, product, _machine_count, item_demand in edges:
            label = f" {item_demand:.3g}/s"
            dot.edge(ing, product, label=label)
        dot.render(str(OUTPUT_DIR_PATH / target), format="png", cleanup=True)

for product in ["engine-unit", "automation-science-pack", "logistic-science-pack", "chemical-science-pack", "military-science-pack", "production-science-pack", "utility-science-pack", "rocket-part", "satellite", "spidertron", "electronic-circuit", "advanced-circuit", "processing-unit"]:
    build_blueprint_graph(recipes, product)
